In [1]:
%matplotlib nbagg
import numpy as np
import matplotlib.pyplot as plt
# load the qcodes path, until we have this installed as a package
import sys

# qcpath = '/Users/Adriaan/Githubrepos/Qcodes'
qcpath = 'D:\GitHubRepos\Qcodes'
if qcpath not in sys.path:
    sys.path.append(qcpath)

# We still need to agree on a good import abreviation, import as q, import as qc? 
import qcodes as qc 
qc.set_mp_method('spawn')  # force Windows behavior on mac

In [2]:
# no extra processes running yet
qc.active_children()

[]

# Making a mock model and experiment. 
This notebook is intended to explain how to make an instrument driver and learn to work with custom parameters. The code in the notebook is based on Qcodes.toymodel used in the QCodes example.ipynb. 

## The toymodel 
In this example we work with a toy model that returns a simple Lorentzian function with some noise. 
It depends on two input parameters, x and y and returns a single value with _output. 

The "ask()" and "write()" functions are there to emulate string based communications similar to a VISA or ethernet socket. 

In [3]:
class ToyLorentzianModel(object): 
    '''
    A simple 2D-Lorentzian. 
    '''
    def __init__(self): 
        self.x = 0
        self.y = 0 
    
    def _output(self): 
        '''
        Returns the 2D lorentzian based on what x and y are set. 
        '''
        Gamma_x = .3 
        Gamma_y = .05
        noise_amplitude = .2
        x0 = .23
        y0 = .21
        xL = self.lor_func(self.x, x0, Gamma_x)
        yL = self.lor_func(self.y, y0, Gamma_y)
        zL = xL + yL + noise_amplitude*np.random.random()
        return zL
        
    def lor_func(self, x, x0=0, Gamma = 1): 
        return Gamma/(2*np.pi*((x-x0)**2+(.5*Gamma)**2))
    
    # Write and ask functions exist only to conform to the structure of the 
    # MockInstrument. It exists to emulate writing strings on VISA or ethernet ports. 
    def ask(self, instrument, parameter): 
        if parameter == 'x':
            return self.x
        elif parameter == 'y':
            return self.y 
        elif parameter =='ampl': 
            return self._output()
        else: 
            raise KeyError('Parameter "%s" not recognized' %parameter)
        
    def write(self, instrument, parameter, value):
        if parameter == 'x': 
            self.x = float(value) # Conversion is required because write passes strings
        elif parameter == 'y':
            self.y = float(value)
        else: 
            raise KeyError('Could not set parameter "%s"' %parameter)

toy_model = ToyLorentzianModel() 


## Example 1, making use of the existing instrument syntax 
1. Make a custom model to talk to for my instruments: 
    2D Lorentzian with noise 
2. Make a single Mock instrument that sets two parameters 
3. Make a mock instrument that gets these parameters 
4. Add it all to a measurement loop 



In [4]:
import inspect # In order to print the docstring in the notebook
from qcodes.instrument.mock import MockInstrument
print('Mock instrument docstring:')
print(MockInstrument.__doc__)

Mock instrument docstring:

    Creates a software instrument, for modeling or testing
    inputs:
        name: (string) the name of this instrument
        delay: the time (in seconds) to wait after any operation
            to simulate communication delay
        model: an object with write and ask methods, taking 2 or 3 args:
            instrument: the name of the instrument
            parameter: the name of the parameter
            value (write only): the value to write, as a string

    parameters to pass to model should be declared with:
        get_cmd = param_name + '?'
        set_cmd = param_name + ' {:.3f}' (specify the format & precision)
    alternatively independent functions may still be provided.
    


## Adding parameters 
The MockInstrument has an add parameter function that adds an instance of the instrument parameter function. 

In [5]:
print('Mock instrument add_parameter docstring (inherited from base.Instrument):')
print(MockInstrument.add_parameter.__doc__)

Mock instrument add_parameter docstring (inherited from base.Instrument):

        binds one InstrumentParameter to this instrument.

        instrument subclasses can call this repeatedly in their __init__
        for every real parameter of the instrument.

        In this sense, parameters are the state variables of the instrument,
        anything the user can set and/or get

        `name` is how the InstrumentParameter will be stored within
        instrument.parameters and also how you  address it using the
        shortcut methods:
        instrument.set(param_name, value) etc.

        see InstrumentParameter for the list of kwargs
        


In [6]:
from qcodes.utils.validators import Numbers # Used to validate inputs
class ToySetInstrument(MockInstrument): 
    '''
    Inherits from the MockInstrument to get all the built in 
    functionality. 
    '''
    def __init__(self, name, **kw):
        super().__init__(name, **kw)
        # Example Mock Instrument makes use of write and ask
        parnames = ['x','y']
        for parname in parnames: 
            self.add_parameter( #adds an instance of the Parameter class 
                name=parname, 
                get_cmd=parname+'?', # Specifies the get command
                set_cmd=parname+' {:.4f}',  # And the set command, {:.4f} specifies the precision
                parse_function=float, # parses the string that get returns 
                vals=Numbers(-100, 100)) # A validator can be used to apply constraints 
            
class ToyGetInstrument(MockInstrument): 
    def __init__(self, name, **kw):
        super().__init__(name, **kw)
        self.add_parameter(
            name='amplitude', 
            get_cmd='ampl'+'?', 
            parse_function=float)   

In [7]:
set_instrument = ToySetInstrument(name='Toy_setter', model=toy_model)
get_instrument = ToyGetInstrument(name='Toy_getter', model=toy_model)

### Testing the get and set functions of the different parameters 

In [8]:
# it's nice to have the key parameters be part of the global namespace
# that way they're objects that we can easily set, get, and slice

# there are multiple ways of addressing a parameter that is part of an instrument
x_par, y_par, ampl_par = set_instrument.parameters['x'],  set_instrument['y'], get_instrument.amplitude
x_par.set(0); y_par.set(0) # Make sure all pars are set to 0
print('Get x and ampl')
print(x_par.get(), ampl_par.get(), '\n')


print('set x and then get x and ampl again')
x_par.set(3.2135643) #printing x_par should demonstrate the rounding to 4 digits
print(x_par.get(), ampl_par.get()) #The amplitude should vary due to the noise I added to the model
print(x_par.get(), ampl_par.get())


Get x and ampl
0.0 0.8771442778541299 

set x and then get x and ampl again
3.2136 0.20098698045810112
3.2136 0.2632376452823068


## Using the toymodel in a Qcodes Loop()

In [9]:
# We add the existing instruments to the qcodes station 
station = qc.Station(set_instrument, get_instrument)
# Define the 'default' measurement (this is a style I would discourage as explicit is better than impl)
station.set_measurement(ampl_par)
station.measure()

[0.3256674977981221]

In [10]:
current_loop = qc.Loop(x_par[-40:40:0.1], 0.03)

In [11]:
qc.active_children()

[]

In [12]:
data=current_loop.run(location='test_sweep_toy')

BrokenPipeError: [Errno 32] Broken pipe


! Note: this example does not exist yet
### Example 2, making custom parameters 
* Set parameter 1 adding a fully custom parameter to instrument 1
* Set parameter 2 a paramter without an instrument that can be set 
* An independent get parameter that acquires the same data but does averaging and returns the mean and sem without modifying the underlying model. 
    

#### Reproduce minimal working example from QCodes example
I verified that the broken pipe error does not occur when reproducing the QCodes example in this notebook (see below) 

In [5]:
# spawn doesn't like function or class definitions in the interpreter
# session - had to move them to a file.
from toymodel import AModel, MockGates, MockSource, MockMeter

# now create this "experiment"
model = AModel()
gates = MockGates('gates', model)
source = MockSource('source', model)
meter = MockMeter('meter', model)

station = qc.Station(gates, source, meter)

# could measure any number of things by adding arguments to this
# function call, but here we're just measuring one, the meter amplitude
station.set_measurement(meter.amplitude)

# it's nice to have the key parameters be part of the global namespace
# that way they're objects that we can easily set, get, and slice
# this could be simplified to a station method that gathers all parameters
# and adds them all as (disambiguated) globals, printing what it did
# something like:
#   station.gather_parameters(globals())
c0, c1, c2, vsd = gates.chan0, gates.chan1, gates.chan2, source.amplitude

# check that a DataServer is running (not yet implemented, but when we have
# a monitor, defining a station will start the DataServer to run the monitor)
qc.active_children()

[]

In [6]:
# start a Loop (which by default runs in a seprarate process)
# the sweep values are defined by slicing the parameter object
# but more complicated sweeps (eg nonlinear, or adaptive) can
# easily be used instead
data = qc.Loop(c0[-20:20:0.1], 0.03).run(location='testsweep')

DataSet: DataMode.PULL_FROM_SERVER, location='testsweep'
   amplitude: amplitude
   chan0: chan0
started at 2016-01-05 09:43:14


In [11]:
data.sync() # If data is not synced the data will not be updated
data.arrays

{'amplitude': DataArray[400]: amplitude
 array([ 0.117,  0.117,  0.115,  0.111,  0.106,  0.099,  0.092,  0.085,
         0.077,  0.071,  0.064,  0.058,  0.053,  0.048,  0.044,  0.04 ,
         0.037,  0.034,  0.031,  0.029,  0.027,  0.025,  0.023,  0.022,
         0.02 ,  0.019,  0.018,  0.017,  0.016,  0.015,  0.014,  0.013,
         0.013,  0.012,  0.011,  0.011,  0.01 ,  0.01 ,  0.01 ,  0.009,
         0.009,  0.008,  0.008,  0.008,  0.007,  0.007,  0.007,  0.007,
         0.006,  0.006,  0.006,  0.006,  0.006,  0.007,  0.007,  0.007,
         0.007,  0.008,  0.008,  0.008,  0.009,  0.009,  0.01 ,  0.01 ,
         0.01 ,  0.011,  0.011,  0.012,  0.013,  0.013,  0.014,  0.015,
         0.016,  0.017,  0.018,  0.019,  0.02 ,  0.022,  0.023,  0.025,
         0.027,  0.029,  0.031,  0.034,  0.037,  0.04 ,  0.044,  0.048,
         0.053,  0.058,  0.064,  0.071,  0.077,  0.085,  0.092,  0.099,
         0.106,  0.111,  0.115,  0.117,  0.117,  0.117,  0.115,  0.111,
         0.106,  0.099, 